In [185]:
from math import inf
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
credentials ={
    'ite': {
        'password':'ITCrowdRules!',
         'url': 'https://iteuropa.circdata-solutions.co.uk/Fusion/login.aspx',
         '__VIEWSTATE' : '/wEPDwUKMTgwMjQwMDI5M2Rksovo2zHemSveuJw8vAuQ6Xf8InA=',
         '__EVENTVALIDATION' : '/wEWBQKPgenHBQLj+sxGAuOoubQBAqqg8cUOAteSgYoCZ204OWUV6uw8bUTN6GYNlbZWM7E='
        },
    'bpl':  {
        'password':'Acumagic10', 
        'url': 'https://bpl.circdata-solutions.co.uk/Fusion/login.aspx',
        '__VIEWSTATE':'/wEPDwUKMTgwMjQwMDI5M2Rk2e1SYhd3ZcfIKWr+jOok9jgKqtU=',
        '__EVENTVALIDATION':'/wEWBQLV2djbAQLj+sxGAuOoubQBAqqg8cUOAteSgYoC7owPjxk9TLLjh/iR0zoJPMA+4Yk=',
}
}

username = 'jubah@bpl-business.com'
company = 'ite'
stats_url = 'https://iteuropa.circdata-solutions.co.uk/Fusion/Dynamail/MailshotDetails.aspx?id=0fe55dd6-105b-4cd6-88d9-17e98105060b'
generic_name_grr = 'CD Weekly - MPU - 15th Aug'
clients = {
    'Union Street': ['#00509f', '#e6aaaa']
}

def isBannedCharacter(character):
    if ord(character) >= 128:
        return True

def remove_characters(string):
    unallowed = ['#', '<', '>', '%', '&', '{', '}', '/', '$', '!', '\'', '\"', ':', '@','*','?','\\','+','+','`','|','=', '-','–' ]
    sanitised_string =''
    for character in range(len(string)):
        if not string[character] in unallowed and not isBannedCharacter(string[character]):
            sanitised_string = sanitised_string + string[character]
    return sanitised_string



def get_basic_info(company, stats_url):

    ## Login to dynamail
    webpage_response = req.get(credentials[company]['url'])
    webpage = webpage_response.content
    # login_page = BeautifulSoup(webpage,'html.parser')

        ##login input fields

    login_creds = {
        'ctl00$CPHP$newloginControl$tbUsername': username,
        'ctl00$CPHP$newloginControl$tbPassword': credentials[company]['password'],
        'ctl00$CPHP$newloginControl$loginButton': 'Login',
        '__VIEWSTATEGENERATOR': '5E82A05A',
        '__VIEWSTATE': credentials[company]['__VIEWSTATE'],
        '__EVENTVALIDATION': credentials[company]['__EVENTVALIDATION'],
    }

    session = req.session()
    res = session.post(credentials[company]['url'], data=login_creds)


    ####################################################################
    #   Go to specific url#######
    stats_page = session.get(stats_url)
    html = './index.html'
    stats_page_soup = BeautifulSoup(stats_page.content, 'html.parser')


    generic_stats ={
        'Sent': [0],
        'Opens':[0],
        'Individual Opens':[0]
    }

    client_info = {
        'subject_line': '',
        'company':''
    }


    ####get total_Sent
    labels = stats_page_soup.find_all(attrs={'class':'Label'})

    for x in labels:

        if(x.string.find('Emails') > 0):   
            generic_stats['Sent'][0] = int(x.parent.find_all('td')[1].find('a').string)
        if(x.string.find('Subject') > 0):
            client_info['subject_line'] = x.parent.find_all('td')[1].string
            print(client_info['subject_line'])
        if(x.string.find('Campaign') > 0):
            client_info['company'] = x.parent.find_all('td')[1].find('a').string
        

    ##### get opened and unique opens
    unique_opens_img = stats_page_soup.find('img', attrs={'id': 'ctl00_CPHP_mailshotCharts_EmailsByStatusChart_Chart1'}).parent.find_all('area')[1].attrs['title']
    unique_opens = int(unique_opens_img[unique_opens_img.index('- ')+1:])
    generic_stats['Individual Opens'][0] = unique_opens

    overall_opens = int(stats_page_soup.find('span', attrs={'id':'ctl00_CPHP_mailshotCharts_OpenedEmailsLabel'}).find('b').text)
    generic_stats['Opens'][0] = overall_opens

    ### santise subject line and subject_line
    start_slice = inf
    end_slice = None
    for x in client_info['subject_line']:
        if(x != ' ' and x !='\n' and x != '\r\n' and x != '\r'):
            end_slice = client_info['subject_line'].rindex(x)
            if(client_info['subject_line'].index(x) < start_slice):
                start_slice = client_info['subject_line'].index(x)
    client_info['subject_line'] = remove_characters(client_info['subject_line'][start_slice:end_slice+1])


    ### santise company and subject_line
    start_slice = inf
    end_slice = None
    for x in client_info['company']:
        if(x != ' ' and x !='\n' and x != '\r\n' and x != '\r'):
            end_slice = client_info['company'].rindex(x)
            if(client_info['company'].index(x) < start_slice):
                start_slice = client_info['company'].index(x)
    client_info['company'] = remove_characters(client_info['company'][start_slice:end_slice+1])

    df = pd.DataFrame(generic_stats)
    df.to_csv(f'{client_info["subject_line"]}.csv', index=False)
    print(df)

    print(generic_stats, client_info)

get_basic_info(company, stats_url=stats_url)



                How do MSPs become carbon neutral? Listen to Julie Bishop â CEO IT Naturally
            
    Sent  Opens  Individual Opens
0  69184   3655              2905
{'Sent': [69184], 'Opens': [3655], 'Individual Opens': [2905]} {'subject_line': 'How do MSPs become carbon neutral Listen to Julie Bishop  CEO IT Naturally', 'company': 'ChannelSec'}
